In [1]:
from econ_lib import data_dir
import pandas as pd

# set up paths
data_dir_aus = data_dir / "AUS_raw_data"
industrial_production_file = data_dir_aus / "5206041_Industry_Indexes.xlsx"
dfd_file = data_dir_aus / "5206004_Expenditure_Price_Indexes.xlsx"
govt_bond_file = data_dir_aus / "govt_bonds_f02hist.xlsx"
govt_bond_historical_file = data_dir_aus / "f02histhist.xls"
corporate_bond_file = data_dir_aus / "corporate_bonds_f03hist.xlsx"
cpi_file = data_dir_aus / "cpi_table_1_640101.xlsx"
cpi_rent_file = data_dir_aus / "cpi_table_13_640111.xlsx"
house_prices_file = data_dir_aus / "QAUR628BIS.xls"
home_ownership_file = data_dir_aus /"1. Housing occupancy and costs, Australia, 1994–95 to 2019–20.xlsx"


In [2]:
# set up date range filters
START_DATE = pd.to_datetime("2005-01-01")
END_DATE = pd.to_datetime("2020-12-31")

# build up individual datasets to produce single dataset covering range
cleaned_data = []

## Industrial Production

In [3]:
df = pd.read_excel(industrial_production_file, sheet_name="Data1")
df = df.set_index(df.columns[0])
METADATA_ENDS_ROW = 10 # row number in excel of the last metadata
metadata = df.head(METADATA_ENDS_ROW-1)
df = df.iloc[METADATA_ENDS_ROW-1:]
SERIES_ID = "A2298671X"

In [4]:
matches = metadata.columns[metadata.loc["Series ID",:]==SERIES_ID]
assert len(matches) ==1
colname = matches[0]
df_industrial_production = df[[colname]].rename(columns={colname:"industrial_production_index"})
df_industrial_production.index.name = "date"
df_industrial_production.index = pd.to_datetime(df_industrial_production.index, format="%Y-%m-%d")
display(df_industrial_production.head(5))
df_industrial_production = df_industrial_production.query("date>=@START_DATE & date<=@END_DATE")


,industrial_production_index
date,
1974-09-01,31.8
1974-12-01,31.4
1975-03-01,29.2
1975-06-01,29.2
1975-09-01,29.7


## Domestic Final Demand

In [5]:
df = pd.read_excel(dfd_file, sheet_name="Data1")
df = df.set_index(df.columns[0])
METADATA_ENDS_ROW = 10 # row number in excel of the last metadata
metadata = df.head(METADATA_ENDS_ROW-1)
df = df.iloc[METADATA_ENDS_ROW-1:]
SERIES_ID = "A2303859F" # IP manufacturing only

In [6]:
matches = metadata.columns[metadata.loc["Series ID",:]==SERIES_ID]
assert len(matches) ==1
colname = matches[0]
df_domestic_final_demand = df[[colname]].rename(columns={colname:"domestic_final_demand_index"})
df_domestic_final_demand.index.name = "date"
df_domestic_final_demand.index = pd.to_datetime(df_domestic_final_demand.index, format="%Y-%m-%d")
display(df_domestic_final_demand.head(5))
df_domestic_final_demand = df_domestic_final_demand.query("date>=@START_DATE & date<=@END_DATE")

,domestic_final_demand_index
date,
1985-09-01,37.3
1985-12-01,38.2
1986-03-01,38.9
1986-06-01,39.4
1986-09-01,40.4


## Government bonds

In [7]:
df = pd.read_excel(govt_bond_file)
df = df.set_index(df.columns[0])
METADATA_ENDS_ROW = 11
metadata = df.head(METADATA_ENDS_ROW-1)
df = df.iloc[METADATA_ENDS_ROW-1:]

In [8]:
SERIES_ID = "FCMYGBAG2"
matches = metadata.columns[metadata.loc["Series ID",:]==SERIES_ID]
assert len(matches) ==1
two_year_colname = matches[0]
SERIES_ID = "FCMYGBAG3"
matches = metadata.columns[metadata.loc["Series ID",:]==SERIES_ID]
assert len(matches) ==1
three_year_colname = matches[0]

In [9]:

df_subset = df[[two_year_colname, three_year_colname]].rename(columns={two_year_colname:"2yr_govt_bond_yield", three_year_colname:"3yr_govt_bond_yield"})
df_subset.index.name = "date"
df_subset.index = pd.to_datetime(df_subset.index, format="%Y-%m-%d")
display(df_subset.head(5))
# cleaned_data.append(df_subset.query("date>=@START_DATE & date<=@END_DATE"))

,2yr_govt_bond_yield,3yr_govt_bond_yield
date,,
2013-05-31,NaN,NaN
2013-06-30,NaN,NaN
2013-07-31,NaN,NaN
2013-08-31,NaN,NaN
2013-09-30,2.688,2.9


### Historical data

In [10]:
df = pd.read_excel(govt_bond_historical_file)
df = df.set_index(df.columns[0])
METADATA_ENDS_ROW = 11
metadata = df.head(METADATA_ENDS_ROW-1)
df = df.iloc[METADATA_ENDS_ROW-1:]

In [11]:
SERIES_ID = "FCMYGBAG2"
matches = metadata.columns[metadata.loc["Mnemonic",:]==SERIES_ID]
assert len(matches) ==1
two_year_colname = matches[0]
SERIES_ID = "FCMYGBAG3"
matches = metadata.columns[metadata.loc["Mnemonic",:]==SERIES_ID]
assert len(matches) ==1
three_year_colname = matches[0]

In [12]:
df_subset_historical = df[[two_year_colname, three_year_colname]].rename(columns={two_year_colname:"2yr_govt_bond_yield", three_year_colname:"3yr_govt_bond_yield"})
df_subset_historical.index.name = "date"
df_subset_historical.index = pd.to_datetime(df_subset_historical.index, format="%Y-%m-%d")
display(df_subset_historical.head(5))

,2yr_govt_bond_yield,3yr_govt_bond_yield
date,,
1969-07-31,NaN,NaN
1969-08-31,NaN,NaN
1969-09-30,NaN,NaN
1969-10-31,NaN,NaN
1969-11-30,NaN,NaN


In [13]:
# check for duplicates
df_subset_historical.index.intersection(df_subset.index)

DatetimeIndex(['2013-05-31'], dtype='datetime64[ns]', name='date', freq=None)

In [14]:
display(df_subset_historical.loc[[pd.to_datetime('2013-05-31')]])
display(df_subset.loc[[pd.to_datetime('2013-05-31')]])


,2yr_govt_bond_yield,3yr_govt_bond_yield
date,,
2013-05-31,2.5225,2.5675


,2yr_govt_bond_yield,3yr_govt_bond_yield
date,,
2013-05-31,NaN,NaN


In [15]:
# drop the nan duplicate and combine
df_subset = df_subset[df_subset.index!=pd.to_datetime('2013-05-31')]
df_govt_bonds_combined = pd.concat([df_subset_historical, df_subset]).query("date>=@START_DATE & date<=@END_DATE")
assert not df_govt_bonds_combined.index.has_duplicates

## Corporate bonds

In [16]:
df = pd.read_excel(corporate_bond_file)
df = df.set_index(df.columns[0])
METADATA_ENDS_ROW = 11
metadata = df.head(METADATA_ENDS_ROW-1)
df = df.iloc[METADATA_ENDS_ROW-1:]
SERIES_ID = "FNFYA3M"

In [17]:
matches = metadata.columns[metadata.loc["Series ID",:]==SERIES_ID]
assert len(matches) ==1
colname = matches[0]
df_subset = df[[colname]].rename(columns={colname:"3yr_corporate_bond_yield"})
df_subset.index.name = "date"
df_subset.index = pd.to_datetime(df_subset.index, format="%Y-%m-%d")
display(df_subset.head(5))
df_corporate_bonds = df_subset.query("date>=@START_DATE & date<=@END_DATE")

,3yr_corporate_bond_yield
date,
2005-01-31,5.95
2005-02-28,6.18
2005-03-31,6.27
2005-04-30,5.94
2005-05-31,5.81


## CPI

In [18]:
df = pd.read_excel(cpi_file, sheet_name="Data1")
df = df.set_index(df.columns[0])
METADATA_ENDS_ROW = 10
metadata = df.head(METADATA_ENDS_ROW-1)
df = df.iloc[METADATA_ENDS_ROW-1:]
SERIES_ID = "A2325846C"

In [19]:
matches = metadata.columns[metadata.loc["Series ID",:]==SERIES_ID]
assert len(matches) ==1
colname = matches[0]
df_subset = df[[colname]].rename(columns={colname:"cpi"})
df_subset.index.name = "date"
df_subset.index = pd.to_datetime(df_subset.index, format="%Y-%m-%d")
display(df_subset.head(5))
df_cpi = df_subset.query("date>=@START_DATE & date<=@END_DATE")

,cpi
date,
1948-09-01,3.7
1948-12-01,3.8
1949-03-01,3.9
1949-06-01,4
1949-09-01,4.1


## CPI - rent

In [20]:
df = pd.read_excel(cpi_rent_file, sheet_name="Data1")
df = df.set_index(df.columns[0])
METADATA_ENDS_ROW = 10
metadata = df.head(METADATA_ENDS_ROW-1)
df = df.iloc[METADATA_ENDS_ROW-1:]
SERIES_ID = "A3604689J"

In [21]:
matches = metadata.columns[metadata.loc["Series ID",:]==SERIES_ID]
assert len(matches) ==1
colname = matches[0]
df_subset = df[[colname]].rename(columns={colname:"rent_index"})
df_subset.index.name = "date"
df_subset.index = pd.to_datetime(df_subset.index, format="%Y-%m-%d")
display(df_subset.head(5))
df_rent = df_subset.query("date>=@START_DATE & date<=@END_DATE")

,rent_index
date,
1972-09-01,9.7
1972-12-01,9.8
1973-03-01,10
1973-06-01,10.1
1973-09-01,10.3


## House prices

In [22]:
df = pd.read_excel(house_prices_file, skiprows=10)
display(df.head(2))
df_house_price_index = df.set_index("observation_date")
df_house_price_index.index.name = 'date'
df_house_price_index.index = pd.to_datetime(df_house_price_index.index)
df_house_price_index.columns = ['house_price_index']
df_house_price_index = df_house_price_index.query("date>=@START_DATE & date<=@END_DATE")
display(df_house_price_index.head())

,observation_date,QAUR628BIS
0,1970-01-01,31.3066
1,1970-04-01,31.5506


,house_price_index
date,
2005-01-01,81.2455
2005-04-01,81.3168
2005-07-01,80.4252
2005-10-01,81.5955
2006-01-01,81.8004


## Home Ownership Rate

In [23]:
df = pd.read_excel(home_ownership_file, sheet_name="Table 1.3", skiprows=4)
df = df.set_index(df.columns[0])
df.index.name = "quantity"
display(df[df.index=='Total owners'])

,Unnamed: 1,1994–95,1995–96,1996–97,1997–98,1999–00,2000–01,2002–03,2003–04,2005–06,2007–08,2009–10,2011–12,2013–14,2015–16,2017–18,2019–20
quantity,,,,,,,,,,,,,,,,,
Total owners,%,71.4,70.9,69.7,70.3,70.6,70.4,69.5,70.0,69.3,68.3,68.8,67.4,67.2,67.5,66.2,66.2
Total owners,MOE(±),1.1,1.1,1.0,1.0,1.2,1.0,1.0,1.0,1.0,1.2,0.8,0.8,1.1,0.9,1.0,1.0


In [26]:
# filter to % and not margin of error values
first_colname = df.columns[0]
total_owners_pct = df[(df.index=='Total owners') & (df[first_colname]=='%')]
total_owners_pct = total_owners_pct.drop(columns=first_colname).T
display(total_owners_pct.head())

total_owners_pct = total_owners_pct.reset_index()
total_owners_pct.columns= ['years', 'home_ownership_pct']
# Note, splitting on wide dash `–` which is not the usual `-`
total_owners_pct['first_year'] = total_owners_pct['years'].str.split("–", expand=True)[0].astype(int)
total_owners_pct['year_start_date'] = pd.to_datetime(total_owners_pct['first_year'], format='%Y')
# shift date to end of year
# from pandas.tseries.offsets import DateOffset
total_owners_pct['year_end_date'] = total_owners_pct['year_start_date'].apply(lambda date : date.replace(month=12, day=31))+pd.DateOffset(years=1)
display(total_owners_pct.head())


quantity,Total owners
1994–95,71.4
1995–96,70.9
1996–97,69.7
1997–98,70.3
1999–00,70.6


,years,home_ownership_pct,first_year,year_start_date,year_end_date
0,1994–95,71.4,1994,1994-01-01,1995-12-31
1,1995–96,70.9,1995,1995-01-01,1996-12-31
2,1996–97,69.7,1996,1996-01-01,1997-12-31
3,1997–98,70.3,1997,1997-01-01,1998-12-31
4,1999–00,70.6,1999,1999-01-01,2000-12-31


### Pad out from 2 yearly to quarterly

In [27]:
# get the dates we want to fill from CPI, except that CPI uses the first day of the final month of the quarter)
display(df_cpi.tail())
target_dates = df_cpi.index + pd.offsets.MonthEnd()
display(target_dates)

,cpi
date,
2019-12-01,116.2
2020-03-01,116.6
2020-06-01,114.4
2020-09-01,116.2
2020-12-01,117.2


DatetimeIndex(['2005-03-31', '2005-06-30', '2005-09-30', '2005-12-31',
               '2006-03-31', '2006-06-30', '2006-09-30', '2006-12-31',
               '2007-03-31', '2007-06-30', '2007-09-30', '2007-12-31',
               '2008-03-31', '2008-06-30', '2008-09-30', '2008-12-31',
               '2009-03-31', '2009-06-30', '2009-09-30', '2009-12-31',
               '2010-03-31', '2010-06-30', '2010-09-30', '2010-12-31',
               '2011-03-31', '2011-06-30', '2011-09-30', '2011-12-31',
               '2012-03-31', '2012-06-30', '2012-09-30', '2012-12-31',
               '2013-03-31', '2013-06-30', '2013-09-30', '2013-12-31',
               '2014-03-31', '2014-06-30', '2014-09-30', '2014-12-31',
               '2015-03-31', '2015-06-30', '2015-09-30', '2015-12-31',
               '2016-03-31', '2016-06-30', '2016-09-30', '2016-12-31',
               '2017-03-31', '2017-06-30', '2017-09-30', '2017-12-31',
               '2018-03-31', '2018-06-30', '2018-09-30', '2018-12-31',
      

In [28]:
df_home_ownership_all_quarters = pd.DataFrame({"date":target_dates})
df_home_ownership_all_quarters = df_home_ownership_all_quarters.merge(total_owners_pct[['home_ownership_pct', 'year_end_date']].rename(columns={"year_end_date":"date"}), how='left', on='date')
display(df_home_ownership_all_quarters.head(6))

,date,home_ownership_pct
0,2005-03-31,NaN
1,2005-06-30,NaN
2,2005-09-30,NaN
3,2005-12-31,NaN
4,2006-03-31,NaN
5,2006-06-30,NaN


In [29]:
df_home_ownership_all_quarters['home_ownership_pct'] = df_home_ownership_all_quarters['home_ownership_pct'].bfill()
display(df_home_ownership_all_quarters.head(6))
df_home_ownership_all_quarters = df_home_ownership_all_quarters.query("date>=@START_DATE & date<=@END_DATE").set_index('date')

,date,home_ownership_pct
0,2005-03-31,69.3
1,2005-06-30,69.3
2,2005-09-30,69.3
3,2005-12-31,69.3
4,2006-03-31,69.3
5,2006-06-30,69.3


## Combine all files
- Check and consolidate how dates are treated for quarters

In [30]:
for df in [df_industrial_production, df_domestic_final_demand, df_govt_bonds_combined,df_corporate_bonds, df_cpi, df_rent, df_house_price_index, df_home_ownership_all_quarters]:
    display(df.head(2))
    print(df.index.has_duplicates)

,industrial_production_index
date,
2005-03-01,65.4
2005-06-01,66.3


False


,domestic_final_demand_index
date,
2005-03-01,69
2005-06-01,69.3


False


,2yr_govt_bond_yield,3yr_govt_bond_yield
date,,
2005-01-31,5.110263,5.144211
2005-02-28,5.4045,5.396


...

False


,house_price_index
date,
2005-01-01,81.2455
2005-04-01,81.3168


False


,home_ownership_pct
date,
2005-03-31,69.3
2005-06-30,69.3


False


In [31]:
df_industrial_production.index = df_industrial_production.index + pd.offsets.MonthEnd()
df_domestic_final_demand.index = df_domestic_final_demand.index + pd.offsets.MonthEnd()
df_cpi.index = df_cpi.index + pd.offsets.MonthEnd()
df_rent.index = df_rent.index + pd.offsets.MonthEnd()
df_house_price_index.index = (df_house_price_index.index - pd.Timedelta(days=1))# + pd.offsets.MonthEnd()

In [32]:
# combined current quarterly datasets minus bond yield data
df_combined_quarterly = pd.concat([df_industrial_production, df_domestic_final_demand, df_cpi, df_rent, df_house_price_index, df_home_ownership_all_quarters], axis=1).sort_index()

In [33]:
display(df_combined_quarterly)

,industrial_production_index,domestic_final_demand_index,cpi,rent_index,house_price_index,home_ownership_pct
date,,,,,,
2004-12-31,NaN,NaN,NaN,NaN,81.2455,NaN
2005-03-31,65.4,69,82.1,70.9,81.3168,69.3
2005-06-30,66.3,69.3,82.6,71.4,80.4252,69.3
2005-09-30,65.8,70,83.4,71.8,81.5955,69.3
2005-12-31,65.6,70.5,83.8,72.3,81.8004,69.3
...,...,...,...,...,...,...
2019-12-31,103,95.4,116.2,112.4,116.3274,66.2
2020-03-31,102.9,95.6,116.6,112.5,116.4502,66.2
2020-06-30,100,95.5,114.4,111,115.5269,66.2


In [34]:
# quick debugging
df_combined_quarterly.to_clipboard()

In [35]:
# Convert date column to year and quarter columns
df_combined_quarterly['year'] = df_combined_quarterly.index.year
df_combined_quarterly['quarter'] = df_combined_quarterly.index.quarter

# Keep only dates from 2005 onwards
df_combined_quarterly = df_combined_quarterly[df_combined_quarterly.index.year >= 2005]

display(df_combined_quarterly)

,industrial_production_index,domestic_final_demand_index,cpi,rent_index,house_price_index,home_ownership_pct,year,quarter
date,,,,,,,,
2005-03-31,65.4,69,82.1,70.9,81.3168,69.3,2005,1
2005-06-30,66.3,69.3,82.6,71.4,80.4252,69.3,2005,2
2005-09-30,65.8,70,83.4,71.8,81.5955,69.3,2005,3
2005-12-31,65.6,70.5,83.8,72.3,81.8004,69.3,2005,4
2006-03-31,65.1,71.1,84.5,72.9,83.3913,69.3,2006,1
...,...,...,...,...,...,...,...,...
2019-12-31,103,95.4,116.2,112.4,116.3274,66.2,2019,4
2020-03-31,102.9,95.6,116.6,112.5,116.4502,66.2,2020,1
2020-06-30,100,95.5,114.4,111,115.5269,66.2,2020,2


In [36]:
# Merge bond yield series
bond_data = pd.concat([df_govt_bonds_combined,df_corporate_bonds], axis=1).sort_index()

# Add credit spread
bond_data['3yr_credit_spread'] = bond_data['3yr_corporate_bond_yield'] - bond_data['3yr_govt_bond_yield']

# Check for rows with missing data
bond_data[bond_data.isnull().any(axis=1)]

,2yr_govt_bond_yield,3yr_govt_bond_yield,3yr_corporate_bond_yield,3yr_credit_spread
date,,,,
2013-06-30,NaN,NaN,4.02,NaN
2013-07-31,NaN,NaN,3.73,NaN
2013-08-31,NaN,NaN,3.9,NaN


In [37]:
# Complete monthly bond yield data from discontinued F3 pdf data (AUS Data/f03hist.pdf)
# This gives data for the corporate bonds and spreads of 1-5 year maturity bonds 

# Approx 3 year credit spread with 1-5 year A bonds spread 
bond_data.at[pd.to_datetime('2013-06-30'), '3yr_credit_spread'] = 1.49
bond_data.at[pd.to_datetime('2013-07-31'), '3yr_credit_spread'] = 1.39
bond_data.at[pd.to_datetime('2013-08-31'), '3yr_credit_spread'] = 1.35

# Approx 2 year govt bond yield as (corporate bond yield - credit spread)
bond_data.at[pd.to_datetime('2013-06-30'), '2yr_govt_bond_yield'] = 4.25-1.49
bond_data.at[pd.to_datetime('2013-07-31'), '2yr_govt_bond_yield'] = 3.97-1.39
bond_data.at[pd.to_datetime('2013-08-31'), '2yr_govt_bond_yield'] = 4.09-1.35

display(bond_data.loc[[pd.to_datetime('2013-06-30'), pd.to_datetime('2013-07-31'), pd.to_datetime('2013-08-31')]])

,2yr_govt_bond_yield,3yr_govt_bond_yield,3yr_corporate_bond_yield,3yr_credit_spread
date,,,,
2013-06-30,2.76,NaN,4.02,1.49
2013-07-31,2.58,NaN,3.73,1.39
2013-08-31,2.74,NaN,3.9,1.35


In [38]:
# Aggregate monthly bond data to quarterly frequency

# Take 3-row averages to aggregate monthly to quarterly data
bond_data['2yr_govt_bond_yield_qrtly'] = bond_data['2yr_govt_bond_yield'].rolling(window=3).mean()
bond_data['3yr_credit_spread_qrtly'] = bond_data['3yr_credit_spread'].rolling(window=3).mean()

# Drop monthly columns and keep only quarterly columns
bond_data = bond_data.drop(columns=['2yr_govt_bond_yield', '3yr_govt_bond_yield', '3yr_corporate_bond_yield', '3yr_credit_spread'])

# Keep rows at the quarter end, i.e. that show averages over a quarter
bond_data = bond_data[bond_data.index.is_quarter_end]

# Convert date column to year and quarter columns
bond_data['year'] = bond_data.index.year
bond_data['quarter'] = bond_data.index.quarter

display(bond_data.head(5))

,2yr_govt_bond_yield_qrtly,3yr_credit_spread_qrtly,year,quarter
date,,,,
2005-03-31,5.365000,0.756993,2005,1
2005-06-30,5.258725,0.585769,2005,2
2005-09-30,5.156753,0.657308,2005,3
2005-12-31,5.299302,0.618323,2005,4
2006-03-31,5.230888,0.691225,2006,1


In [39]:
df_combined = pd.merge(df_combined_quarterly, bond_data, on=['year', 'quarter'], how='inner')

# List of columns with 'year' and 'quarter' first
new_column_order = ['year', 'quarter', 'cpi', 'industrial_production_index', 'domestic_final_demand_index', '2yr_govt_bond_yield_qrtly', '3yr_credit_spread_qrtly', 'house_price_index', 'rent_index', 'home_ownership_pct']

# Reorder the DataFrame
df_combined = df_combined[new_column_order]
display(df_combined.head(5))

,year,quarter,cpi,industrial_production_index,domestic_final_demand_index,2yr_govt_bond_yield_qrtly,3yr_credit_spread_qrtly,house_price_index,rent_index,home_ownership_pct
0,2005,1,82.1,65.4,69,5.365000,0.756993,81.3168,70.9,69.3
1,2005,2,82.6,66.3,69.3,5.258725,0.585769,80.4252,71.4,69.3
2,2005,3,83.4,65.8,70,5.156753,0.657308,81.5955,71.8,69.3
3,2005,4,83.8,65.6,70.5,5.299302,0.618323,81.8004,72.3,69.3
4,2006,1,84.5,65.1,71.1,5.230888,0.691225,83.3913,72.9,69.3


In [40]:
# Write the cleaned data to an Excel file in the data directory for the scripts to use
out_dir = data_dir #/"cleaned"
out_dir.mkdir(parents=True, exist_ok=True)
df_combined.to_excel(out_dir /"quarterly_data_AUS.xlsx", index=False)

## MP Shocks

In [47]:
pdata_filepath = data_dir_aus / "mp-surprises.csv"
pdata = pd.read_csv(pdata_filepath)
pdata

pdata['decision_date'] = pd.to_datetime(pdata['decision_date'])
pdata = pdata.drop(columns=['decision_date_time', 'cashrate_change'])
pdata = pdata.set_index(pdata.columns[0])

pdata


,pc1_scaled
decision_date,
2001-04-04,-0.109710
2001-05-02,0.039571
2001-06-06,0.055921
2001-07-04,-0.005033
2001-08-08,0.014644
...,...
2020-02-04,0.058728
2020-03-03,0.031763
2020-03-19,0.019607


In [48]:
# Aggregate to monthly shocks to quarterly

# Ensure 'year' and 'quarter' columns are present
pdata['year'] = pdata.index.year
pdata['quarter'] = pdata.index.quarter

# Group by year and quarter, then calculate the mean shock for each quarter
pdata['agg_shock'] = pdata.groupby(['year', 'quarter'])['pc1_scaled'].transform(lambda x: x.rolling(window=2).mean() if len(x) == 2 else x.rolling(window=3).mean())


# Keep quarter end rows only
pdata = pdata.dropna(subset=['agg_shock'])
pdata = pdata[['year', 'quarter', 'agg_shock']]
pdata.reset_index(drop=True, inplace=True)
pdata

,year,quarter,agg_shock
0,2001,2,-0.004739
1,2001,3,-0.011947
2,2002,4,-0.002258
3,2003,1,0.020357
4,2003,2,0.016914
...,...,...,...
68,2019,2,0.042845
69,2019,3,0.003174
70,2019,4,-0.009729
71,2020,1,0.036699


In [49]:
# Write the cleaned data to an Excel file in the data directory for the scripts to use
pdata.to_excel(data_dir /"quarterly_shocks_AUS.xlsx", index=False)